In [1]:
import  json
from os import listdir
from os.path import isfile, join
import csv
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re

Load the data from JSON

In [ ]:
data = json.load(open('rumoureval-subtaskA-train.json'))

dev = json.load(open('rumoureval-subtaskA-dev.json'))



Match the respective tweets by ID

In [ ]:
csv = open('traindata.csv', "w", encoding='utf-8-sig')
columnTitleRow = "ID,tweet_text,results\n"
csv.write(columnTitleRow)

for k,v in data.items():
    json_path=""
    mypath="C:/Users/Nicole/semeval2017-task8-dataset/rumoureval-data/"
    
    for d in listdir(mypath):
        for thread in listdir(mypath+d):
            if k == thread:
                json_path=mypath+d+"/"+thread+"/source-tweet/"+k+".json"
                break
            else:                
                reply_path=mypath+d+"/"+thread+"/replies/"
                if k+".json" in listdir(reply_path):
                    json_path=reply_path+k+".json"
                    break
        if json_path!="":
            break
    if (isfile(json_path)):
            source = json.load(open(json_path, 'r'))
            print(json_path)
    else:
        print ("False:" + k)
        print(json_path)
        print("end\n")
    #print (source)
    reply_tweet = source['text']
    reply_tweet = reply_tweet.replace("\n", " ")
    reply_tweet = reply_tweet.replace("\"", "\"\"")
    row = k + "," + "\"" + reply_tweet + "\"" + "," + v + "\n"
    csv.write(row)
    #print (source['text'])
    #print (v+"\n")
    

In [ ]:
dataset = pd.read_csv('traindata.csv', error_bad_lines=False)

In [ ]:
dataset.head()

In [ ]:
csv.close()

Information of the training data

In [ ]:
dataset.info()

use the following code to see if the data is loaded right

In [ ]:
dataset[dataset.isnull().any(axis=1)]

In [ ]:
np.sum(dataset.isnull().any(axis=1))

In [ ]:
dataset.isnull().any(axis=0)

In [ ]:
dataset = dataset.rename(columns={" tweet_text": "tweet_text", " results": "results"})

the tweets of different classes

In [ ]:
dataset['results'].value_counts()

The data is skew

we will need to try oversampling the 3 groups other than 'comment' group, devide them into 4 groups

In [ ]:
dfcomment = dataset[dataset.results == 'comment']

In [ ]:
dfsupport = dataset[dataset.results == 'support']

In [ ]:
dfdeny = dataset[dataset.results == 'deny']

In [ ]:
dfquery = dataset[dataset.results == 'query']

In [ ]:
dfquery.head()

# data cleansing

In [ ]:
dataset['text_len'] = [len(t) for t in dataset.tweet_text]

In [ ]:
dataset.head()

In [ ]:
fig, ax = plt.subplots(figsize=(5, 5))
plt.boxplot(dataset.text_len)
plt.show()

In [ ]:
testing = dataset[dataset['text_len']>140]

In [ ]:
testing.head()

tweet has length limit with 140, to see why they exceed 140

In [ ]:
testing['tweet_text'].iloc[1]

there are some HTML decoding problem

In [ ]:
len(testing['tweet_text'])

# HTML decoding

use the beautiful soup to do the html decoding

In [ ]:
from bs4 import BeautifulSoup

run a test on it

In [ ]:
example1 = BeautifulSoup(testing['tweet_text'].iloc[3], 'html.parser')
example1.get_text()

In [ ]:
example1

In [ ]:
# def tweet_cleaner(text):
#     example1 = BeautifulSoup(text, 'html.parser')
#     return example1.get_text()
    

In [ ]:
clean_tweet_texts = []
for i in dataset['tweet_text']:
    clean_tweet_texts.append(BeautifulSoup(i, 'html.parser').get_text())

In [ ]:
len(clean_tweet_texts)

In [ ]:
clean_df = pd.DataFrame(clean_tweet_texts,columns=['text'])
clean_df['results'] = dataset['results']
clean_df['ID'] = dataset['ID']
clean_df.head()

In [ ]:
htmldata.info()

In [ ]:
clean_df.to_csv('training_data_with_html_decode.csv', index = None)

In [2]:
htmldata = pd.read_csv('training_data_with_html_decode.csv')

In [5]:
htmldata

,text,results,ID
0,@max843 @DanteB4u @Alasscan_ This shows Micha...,comment,5.017606e+17
1,@USATODAY Get ready to tweet the address #Spik...,comment,5.002702e+17
2,@rikermacdaniels @CBCAlerts @CBCNews @JustinTr...,comment,5.249712e+17
3,@RT_com Just kill them.,comment,5.528369e+17
4,"@TheAnonMessage2 no, because that would requir...",support,5.002899e+17
5,@Simon_Cullen @HalaJaber AU GOV and Tony Abbot...,comment,5.442932e+17
6,@Kiippyy @KEEMSTARx u know theres some fucked ...,comment,5.442830e+17
7,@jlpauk why would the ferguson PD and STLCO PD...,comment,5.004067e+17
8,@140elect Here are pixs I have. http://t.co/jn...,comment,5.003002e+17
9,@RT_com @SpringboroTea http://t.co/5NjSg1pBSi,comment,5.528116e+17


In [6]:
htmldata.head()

,text,results,ID
0,@max843 @DanteB4u @Alasscan_ This shows Micha...,comment,5.017606e+17
1,@USATODAY Get ready to tweet the address #Spik...,comment,5.002702e+17
2,@rikermacdaniels @CBCAlerts @CBCNews @JustinTr...,comment,5.249712e+17
3,@RT_com Just kill them.,comment,5.528369e+17
4,"@TheAnonMessage2 no, because that would requir...",support,5.002899e+17


check the length again

In [ ]:
htmldata['text_len'] = [len(t) for t in htmldata.text]

In [ ]:
htmldata[htmldata['text_len']>140]

it's empty so we're good to go

In [ ]:
htmldata['text'].iloc[0]

In [ ]:
htmldata['text'].iloc[4217]

In [ ]:
htmldata['text'].iloc[9]

for pre-processing, there's a Python library which is used to deal with the tweets preprocessing, it will clean out:
URLs
Hashtags
Mentions

Emojis
Smileys

I decided to remove the URLs, Mentions,  Emojis, Smileys, numbers, but for hashtags, I will only remove the hashtag symbol and keep the word following the hashtag symbol as some of it is the part of the sentense and it's useful, so I'm not using the hashtags option here, and I will remove the Reserved words (RT, FAV) in the tweets

In [7]:
import preprocessor as p

In [8]:
p.set_options(p.OPT.URL, p.OPT.EMOJI, p.OPT.MENTION, p.OPT.RESERVED, p.OPT.NUMBER, p.OPT.SMILEY)

In [ ]:
p.clean('@max843 @DanteB4u @Alasscan_ This shows :)')

In [ ]:
re.sub(' RT|#', ' ', p.clean('Preprocessor is #awesome 👍 https://github.com/s/preprocessor RTl'))

In [ ]:
re.sub(' RT|#', '', ' RT Get ready to tweet the address #Spik')

make small tests first

In [ ]:
p.clean(htmldata['text'].iloc[0])

In [ ]:
re.sub(' RT', ' ', p.clean(htmldata['text'].iloc[0]))

In [ ]:
htmldata['text'].iloc[4217]

In [ ]:
p.clean(htmldata['text'].iloc[4217])

remove punctuations, non-alphabaet characters and separates the word with a single space

In [ ]:
' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)"," ",testtext).split())

do it on all the data

In [9]:
clean_tweet_texts1 = []
for i in htmldata['text']:
    clean_tweet_texts1.append(re.sub(' RT|#', '',p.clean(i))) #remove 'RT' and hashtags

In [ ]:
# clean_tweet_texts2 = []
# for i in clean_tweet_texts1:
#     clean_tweet_texts2.append(' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)"," ",i).split()))

In [10]:
clean_tweet_texts2_df = pd.DataFrame(clean_tweet_texts1,columns=['text'])
clean_tweet_texts2_df['results'] = htmldata['results']
clean_tweet_texts2_df['ID'] = htmldata['ID']
clean_tweet_texts2_df.head()

,text,results,ID
0,This shows Michael paid for cigarillos. No rob...,comment,5.017606e+17
1,Get ready to tweet the address SpikeLee and ot...,comment,5.002702e+17
2,your radical Muslims are being funded by Ameri...,comment,5.249712e+17
3,Just kill them.,comment,5.528369e+17
4,"no, because that would require reasoning and t...",support,5.002899e+17


In [11]:
clean_tweet_texts2_df['text'].iloc[0]

'This shows Michael paid for cigarillos. No robbery, no crime! Pls'

In [12]:
clean_tweet_texts2_df.to_csv('clean_trainingdata.csv', index = None)

In [13]:
df = pd.read_csv('clean_trainingdata.csv')

In [ ]:
df = clean_tweet_texts2_df

In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4239 entries, 0 to 4238
Data columns (total 3 columns):
text       4139 non-null object
results    4238 non-null object
ID         4238 non-null float64
dtypes: float64(1), object(2)
memory usage: 99.4+ KB


In [15]:
df[df.isnull().any(axis=1)]

,text,results,ID
9,NaN,comment,5.528116e+17
29,NaN,comment,5.445206e+17
62,NaN,comment,5.249378e+17
64,NaN,query,5.535916e+17
95,NaN,comment,5.445189e+17
112,NaN,comment,5.535353e+17
150,NaN,comment,5.249336e+17
281,NaN,comment,5.442908e+17
375,NaN,comment,5.443494e+17
380,NaN,comment,5.445204e+17


In [ ]:
dataset.iloc[62]

In [18]:
htmldata.iloc[1108]

text       @Slate @jasonCwaite http://t.co/FDMCU0siks
results                                          deny
ID                                        5.52851e+17
Name: 1108, dtype: object

remove those empty texts

In [19]:
df.dropna(inplace=True)

In [ ]:
df.info()

In [20]:
df.to_csv('clean_trainingdata.csv', index = None)

In [21]:
df['results'].value_counts()

comment    2661
support     824
deny        328
query       325
Name: results, dtype: int64

In [22]:
dfcomment = df[df.results == 'comment']
dfsupport = df[df.results == 'support']
dfdeny = df[df.results == 'deny']
dfquery = df[df.results == 'query']

In [38]:
newdf1 = pd.concat([dfquery, dfquery])

In [45]:
len(newdf2)

2472

In [36]:
newdf = pd.concat([newdf, newdf]) #oversample the deny

In [43]:
newdf1 = pd.concat([newdf1, newdf1]) #oversample the query

In [24]:
newdf2 = pd.concat([newdf2, dfsupport]) #oversample the support

In [46]:
newdataset = pd.concat([newdf2, dfcomment,newdf,newdf1])

In [47]:
newdataset.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10357 entries, 4 to 4203
Data columns (total 3 columns):
text       10357 non-null object
results    10357 non-null object
ID         10357 non-null float64
dtypes: float64(1), object(2)
memory usage: 323.7+ KB


In [48]:
newdataset.to_csv('new_trainingdata.csv', index = None)

In [54]:
newdataset = pd.read_csv('new_trainingdata.csv')

In [ ]:
newdataset.head()

In [ ]:
newdataset.info()

tokenize the texts

In [56]:
from nltk.tokenize import TweetTokenizer

In [57]:
tknzr = TweetTokenizer(strip_handles=True, reduce_len=True)

In [58]:
temp = [] 
for x in newdataset['text']:
    temp.append(tknzr.tokenize(x))
temp = pd.Series(temp)
newdataset['reply_text'] = temp

In [59]:
newdataset

,text,results,ID,reply_text
0,"no, because that would require reasoning and t...",support,5.002899e+17,"[no, ,, because, that, would, require, reasoni..."
1,It is according to this terrorism eert,support,5.443078e+17,"[It, is, according, to, this, terrorism, eert]"
2,"The day Ferguson cops told a dirty, bloody lie...",support,5.003198e+17,"[The, day, Ferguson, cops, told, a, dirty, ,, ..."
3,Several hostages freed at Jewish supermarket i...,support,5.535891e+17,"[Several, hostages, freed, at, Jewish, superma..."
4,Shooter still on loose after uniformed soldier...,support,5.249257e+17,"[Shooter, still, on, loose, after, uniformed, ..."
5,well said,support,5.249647e+17,"[well, said]"
6,my thoughts also,support,5.249391e+17,"[my, thoughts, also]"
7,"I could not agree more, it just plain sounds p...",support,5.002815e+17,"[I, could, not, agree, more, ,, it, just, plai..."
8,The Canadian soldier killed in today's Ottawa ...,support,5.250604e+17,"[The, Canadian, soldier, killed, in, today's, ..."
9,Five Hostages Escape Sydney Cafe Siege — Live ...,support,5.443829e+17,"[Five, Hostages, Escape, Sydney, Cafe, Siege, ..."


shuffle the data

In [ ]:
newdataset.sample(len(newdataset))

Found that some abbreviation are tokenized incorrectly, so we need to go back and modify the cleansing process, 